In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train_hp.csv')
train_df.head()



In [ ]:
test_df = pd.read_csv('../input/test_hp.csv')
test_df.head()

In [ ]:
train_df = train_df.drop(['User_ID'],axis=1)
train_df.Browser_Used.unique()

In [ ]:
train_df.Browser_Used.unique()


In [ ]:
test_df.Browser_Used.unique()


In [ ]:
train_df.Device_Used.unique()
train_df.head()


In [ ]:
test_df.Device_Used.unique()
test_df.head()


In [ ]:
ip_addresses = train_df.Browser_Used.unique()
ip_dict1 = dict(zip(ip_addresses, range(len(ip_addresses))))
train_df = train_df.replace(ip_dict1)

ip_addresses = train_df.Device_Used.unique()
ip_dict2 = dict(zip(ip_addresses, range(len(ip_addresses))))
train_df = train_df.replace(ip_dict2)

train_df.head()

In [ ]:
ip_addresses = test_df.Browser_Used.unique()
ip_dict1 = dict(zip(ip_addresses, range(len(ip_addresses))))
test_df = test_df.replace(ip_dict1)

ip_addresses = test_df.Device_Used.unique()
ip_dict2 = dict(zip(ip_addresses, range(len(ip_addresses))))
test_df = test_df.replace(ip_dict2)

test_df.head()

In [ ]:
test_df['Browser_Used'] = test_df['Browser_Used']+1
test_df['Device_Used'] = test_df['Device_Used']+1
test_df.head()

In [ ]:
train_df['Browser_Used'] = train_df['Browser_Used']+1
train_df['Device_Used'] = train_df['Device_Used']+1
train_df.head()

In [ ]:
ip_addresses = train_df.Is_Response.unique()
ip_dict3 = dict(zip(ip_addresses, range(len(ip_addresses))))
train_df = train_df.replace(ip_dict3)
train_df.head()

In [ ]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
cvec = CountVectorizer(stop_words='english', min_df=.0015, max_df=.4, ngram_range=(1,2))
cvec

In [ ]:
cvec.fit(train_df['Description'].values)

In [ ]:
len(cvec.vocabulary_)

In [ ]:
cvec_counts = cvec.transform(train_df['Description'])
print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

In [ ]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

In [ ]:
# using the TfidfTransformer to calculate the weights for each term in each document 

transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

In [ ]:
# top 20 terms by average tf-idf weight:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)



In [ ]:
# using the TfidfVectorizer class
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0015, max_df=.4, stop_words='english', ngram_range=(1,2))
tvec_weights = tvec.fit_transform(train_df['Description'])
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)



In [ ]:
tvec_weights.shape

In [ ]:
X_train = tvec_weights[:35000]
y_train = train_df['Is_Response'][:35000]

X_test = tvec_weights[35000:]
y_test = train_df['Is_Response'][35000:]


In [ ]:
# Naive bayes Model
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)



predicted = clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)


predicted_RandomForestClassifier = classifier.predict(X_test)
np.mean(predicted_RandomForestClassifier == y_test)

In [ ]:
# using xgboost classifier
from xgboost import XGBClassifier
# fit model into training data
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
np.mean(y_pred == y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
smreg_model = LogisticRegression(multi_class="multinomial", solver='lbfgs')

smreg_model.fit(X_train, y_train)

predicted_LR = smreg_model.predict(X_test)
np.mean(predicted_LR == y_test)


In [ ]:
test_df.head()

In [ ]:
cvec_t_counts = cvec.transform(test_df['Description'])

In [ ]:
len(cvec.vocabulary_)

In [ ]:
cvec_t_counts = cvec.transform(test_df['Description'])

In [ ]:
print('sparse matrix shape:', cvec_t_counts.shape)

In [ ]:
X_Test=cvec_t_counts

In [ ]:
from sklearn.linear_model import LogisticRegression
smreg_model = LogisticRegression(multi_class="multinomial", solver='lbfgs')

smreg_model.fit(X_train, y_train)

predicted_LR = smreg_model.predict(X_Test)


sub = pd.DataFrame()
sub['User_ID'] = test_df['User_ID']
sub['Is_Response'] = predicted_LR
sub

dict = {1:'happy',0:'not_happy'}
sub = sub.replace(dict)
sub

sub.to_csv('output.csv')